In [4]:
import stan
import nest_asyncio

In [2]:
schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

In [3]:
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

Note this is needed :
https://pystan.readthedocs.io/en/latest/faq.html#how-can-i-use-pystan-with-jupyter-notebook-or-jupyterlab


In [9]:
nest_asyncio.apply()


In [13]:
posterior = stan.build(schools_code, data=schools_data)
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires panda

Building: found in cache, done.
Messages from stanc:
Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.18 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.7e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.17 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.9e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.19 seconds.
  Adjust your expectations accordingly!
/Users/ronaldlegere/opt/anaconda3/envs/BayesRules/lib/python3.7/site-packages/pandas/_libs/

In [16]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parameters,,,,,,,,
lp__,4000.0,-39.487577,2.568259,-5.350165e+01,-41.073062,-39.274072,-37.667005,-32.833059
accept_stat__,4000.0,0.835352,0.236494,1.702209e-71,0.789436,0.945044,0.985260,1.000000
stepsize__,4000.0,0.352327,0.067767,2.807955e-01,0.308029,0.333322,0.377620,0.461870
treedepth__,4000.0,3.400250,0.574572,1.000000e+00,3.000000,3.000000,4.000000,5.000000
n_leapfrog__,4000.0,11.497000,4.234793,1.000000e+00,7.000000,15.000000,15.000000,31.000000
divergent__,4000.0,0.001000,0.031611,0.000000e+00,0.000000,0.000000,0.000000,1.000000
energy__,4000.0,44.502827,3.448663,3.480742e+01,42.053190,44.240091,46.627510,62.668609
mu,4000.0,7.950235,5.201871,-1.133945e+01,4.661105,7.861669,11.158651,26.809430
tau,4000.0,6.755982,5.862831,5.107209e-03,2.512304,5.268053,9.263765,41.103115
